In [1]:
# import realted module
import pandas as pd
import numpy as np

# read the file
leave16= pd.read_csv('F:/leave16.csv',header=None, delimiter=',', 
                     names = ['dayofservice','tripid','progrnumber','stoppointid','plannedtime_arr','plannedtime_dep','actualtime_arr','actualtime_dep','suppressed'])

In [2]:
# Check the rows and columns number of the table
leave16.shape

(56843319, 9)

In [3]:
# Get an overview of the table
leave16.head()

,dayofservice,tripid,progrnumber,stoppointid,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,suppressed
0,2016-01-31,2811044,63,3390,41820,41820,42920,42920,NaN
1,2016-01-31,2820852,1,7158,69300,69300,69320,69320,NaN
2,2016-01-31,2820852,5,7017,69498,69498,69532,69532,NaN
3,2016-01-31,2820852,10,1893,69711,69711,69858,69858,NaN
4,2016-01-31,2820853,16,1648,76822,76822,76636,76657,NaN


In [4]:
# Read the trips file
trips16 = pd.read_csv('F:/trips16.csv', delimiter=',')

# Check the shape of the table
trips16.shape

(928098, 10)

In [5]:
# Get an overview of the trips table
trips16.head()

,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,suppressed
0,2016-02-09,2824642,145,145_105,2,45668,40800,45375.0,40782.0,NaN
1,2016-02-09,2826717,9,9_7,2,65553,60600,66258.0,60582.0,NaN
2,2016-02-09,2826730,54A,54A_12,2,65951,62100,66053.0,62078.0,NaN
3,2016-02-09,2826743,7,7_51,1,54763,50400,54854.0,50383.0,NaN
4,2016-02-09,2812908,39,39_20,1,27375,22920,27318.0,22931.0,NaN


In [6]:
# Read the weather data
weather16 = pd.read_csv('2016weatherClean.csv', delimiter=',')
# weather16['dt'] = pd.to_datetime(weather16['dt'],unit='s')
weather16 = weather16.sort_values('dt', ascending=False)
weather16 = weather16.reindex(index=weather16.index[::-1])

Here we won't add all features of the weather data to our model. We just select some important ones, including the followings:
- temp
- pressure
- humidity
- wind_speed
- weather_description
- clouds_all
- wind_deg

In [7]:
weather_related = weather16[['dt','temp','pressure','humidity','wind_speed','weather_description','clouds_all','wind_deg']]
weather_related.head()

,dt,temp,pressure,humidity,wind_speed,weather_description,clouds_all,wind_deg
0,1451606400,275.15,1015,80,4,scattered clouds,40,200
1,1451613600,274.15,1016,80,2,scattered clouds,40,170
2,1451620800,276.15,1015,69,3,scattered clouds,40,130
3,1451628000,277.09,1014,69,4,scattered clouds,40,150
4,1451631600,277.09,1013,100,3,broken clouds,75,150


In [8]:
merge_table =  pd.merge(trips16 , leave16, on = ['tripid', 'dayofservice'], how = 'inner')
merge_table.shape

(47479499, 17)

In [9]:
merge_table.head(5)

,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr_x,plannedtime_dep_x,actualtime_arr_x,actualtime_dep_x,suppressed_x,progrnumber,stoppointid,plannedtime_arr_y,plannedtime_dep_y,actualtime_arr_y,actualtime_dep_y,suppressed_y
0,2016-02-09,2824642,145,145_105,2,45668,40800,45375.0,40782.0,NaN,11,2997,41506,41506,41403,41426,NaN
1,2016-02-09,2824642,145,145_105,2,45668,40800,45375.0,40782.0,NaN,10,4151,41452,41452,41367,41384,NaN
2,2016-02-09,2824642,145,145_105,2,45668,40800,45375.0,40782.0,NaN,9,4182,41405,41405,41319,41332,NaN
3,2016-02-09,2824642,145,145_105,2,45668,40800,45375.0,40782.0,NaN,8,4181,41314,41314,41283,41283,NaN
4,2016-02-09,2824642,145,145_105,2,45668,40800,45375.0,40782.0,NaN,7,4180,41243,41243,41238,41251,NaN


In [10]:
output = merge_table[['dayofservice','tripid','lineid','routeid','direction','progrnumber','stoppointid','plannedtime_arr_y','plannedtime_dep_y','actualtime_arr_y','actualtime_dep_y','suppressed_y']]

In [11]:
output.shape

(47479499, 12)

In [12]:
# Select rows with lineid == 46A and direction == 1 
trips16_46A = trips16.loc[(trips16['lineid'] == '46A') & (trips16['direction'] == 1)]
trips16_46A.head(10)

,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,suppressed
103,2016-02-09,2815441,46A,46A_74,1,62257,57600,62587.0,57639.0,NaN
154,2016-02-09,2822373,46A,46A_74,1,33599,28320,33931.0,28323.0,NaN
199,2016-04-08,3010331,46A,46A_74,1,77695,74400,77565.0,74375.0,NaN
223,2016-04-02,2902305,46A,46A_74,1,66140,61800,66544.0,62003.0,NaN
304,2016-04-02,2905183,46A,46A_74,1,53647,49200,53670.0,49630.0,NaN
330,2016-04-02,2909721,46A,46A_74,1,51847,47400,51699.0,47370.0,NaN
366,2016-04-07,3018107,46A,46A_74,1,61906,57120,62097.0,56944.0,NaN
367,2016-04-07,3014232,46A,46A_74,1,60946,56160,61634.0,56200.0,NaN
368,2016-04-07,3004076,46A,46A_74,1,63346,58560,63126.0,58529.0,NaN
369,2016-04-07,3008876,46A,46A_74,1,62386,57600,62098.0,57579.0,NaN


In [13]:
# Join the trip table and leavetime table, use inner jopz
out = pd.merge(trips16_46A , leave16, on = ['tripid', 'dayofservice'], how = 'inner')

In [14]:
out.shape

(889694, 17)

In [15]:
out['routeid'].unique()

array(['46A_74', '46A_68', '46A_70', '46A_69', '46A_73', '46A_72'],
      dtype=object)

In [16]:
out['comb'] = out['dayofservice'].map(str) + out['tripid'].map(str)
u = out['comb'].unique()

In [19]:
len(u)

16017

Features:
- start_stop
- end_stop
- date
- weather

Target Value:
- Time. (The dearture time of end - The departure time of start)

Notes: The start_stop and end_stop is adjancent.

In [20]:
trip_46A = out['tripid'].unique()
date_46A = out['dayofservice'].unique()

result = pd.DataFrame(columns = ['dt','dayofweek','month','day','arrive_time','start_point','end_point','duration'])


# for trip in trip_46A:
#     for date in date_46A:
for date_trip in u:
        day_trip = out.loc[(out['comb'] == date_trip)].sort_values(by=['progrnumber'])
        try:
            day_trip.index = range(len(day_trip))

            # Calculate the time between each pair stops
            time = day_trip['actualtime_arr_y'].shift(-1) - day_trip['actualtime_arr_y']
            time.drop(time.tail(1).index,inplace=True)

            # Get the month and the day, month and the day of week
            month = pd.to_datetime(day_trip['dayofservice']).dt.month
            day = pd.to_datetime(day_trip['dayofservice']).dt.day
            dayofweek = pd.to_datetime(day_trip['dayofservice']).dt.dayofweek

            # Add datetime and actual_arrive time as current unix time
            datetime = pd.DatetimeIndex(day_trip['dayofservice']).astype(np.int64)/1000000000 + day_trip['actualtime_arr_y']
            datetime = pd.DataFrame(data={'unixtime':datetime})

            # Conver float to int, this is for the following merge operation
            datetime['unixtime'] = datetime['unixtime'].astype(np.int64)

            # Set End point 
            end_point = day_trip['stoppointid'].shift(-1)
            end_point.drop(end_point.tail(1).index,inplace=True)

            # Merge these columns
            a = pd.concat([datetime,dayofweek, month, day, day_trip['actualtime_arr_y'], day_trip['stoppointid'], end_point, time], axis = 1)

            # Change the name of columns
            a.columns = ['dt','dayofweek','month','day','arrive_time','start_point','end_point','duration']
            a.drop(a.tail(1).index,inplace=True)

            # Merge two tables
            r = pd.merge_asof(a, weather_related, on = "dt",direction='nearest')

            result = pd.concat([r, result])
            
        except:
            pass
        
# result

In [21]:
result.to_csv('result_of_46A.csv')

In [22]:
result.shape

(871584, 15)